In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision.transforms import InterpolationMode

from going_modular.model.TripletFaceRecognition import EmbeddingNet, TripletNet
from going_modular.dataloader.triplet import TripletDataset, GaussianNoise, RandomResizedCropRect
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.train_eval.triplet.train import fit
from going_modular.dataloader.triplet import CustomExrDataset
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

import os
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'epochs': 400,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}

train_transform = transforms.Compose([
    RandomResizedCropRect(size=CONFIGURATION['image_size']),
    GaussianNoise(),
])

test_transform = transforms.Compose([
    transforms.Resize((CONFIGURATION['image_size'], CONFIGURATION['image_size']), interpolation=InterpolationMode.BILINEAR),
])

triplet_train_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=train_transform, train=True)
triplet_test_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], transform=test_transform, train=False)

triplet_train_loader = DataLoader(
    triplet_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

triplet_test_loader = DataLoader(
    triplet_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

roc_train_dataset = CustomExrDataset(CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'])
roc_train_loader = DataLoader(
    roc_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_test_dataset = CustomExrDataset(CONFIGURATION['test_dir'], transform=test_transform, type=CONFIGURATION['type'])
roc_test_loader = DataLoader(
    roc_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

embedding_net = EmbeddingNet(len_embedding=CONFIGURATION['embedding_size']).to(device)
model = TripletNet(embedding_net).to(device)
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models')
early_min_stopping = MultiMetricEarlyStopping(
    monitor_keys=['test_loss'],
    patience=50,
    mode='min',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=10
)      

early_max_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'cosine_acc', 'euclidean_auc', 'euclidean_acc'],
    patience=50,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=20
)      

checkpoint_path = os.path.abspath('checkpoint/triplet/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

2024-12-19 14:21:08.415261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 14:21:09.703979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device, 
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_max_stopping=early_max_stopping,
    early_min_stopping=early_min_stopping,
    model_checkpoint=modle_checkpoint
)

start train


Processing batch 1/52
Batch 1: Anchors shape torch.Size([16, 512])
Batch 1: Loss computed
Batch 1: Loss value 157.66543579101562
Batch 1: Backward pass done
Batch 1: Optimizer step done
Processing batch 2/52
Batch 2: Anchors shape torch.Size([16, 512])
Batch 2: Loss computed
Batch 2: Loss value 106.80268859863281
Batch 2: Backward pass done
Batch 2: Optimizer step done
Processing batch 3/52
Batch 3: Anchors shape torch.Size([16, 512])
Batch 3: Loss computed
Batch 3: Loss value 81.90013122558594
Batch 3: Backward pass done
Batch 3: Optimizer step done
Processing batch 4/52
Batch 4: Anchors shape torch.Size([16, 512])
Batch 4: Loss computed
Batch 4: Loss value 51.162384033203125
Batch 4: Backward pass done
Batch 4: Optimizer step done
Processing batch 5/52
Batch 5: Anchors shape torch.Size([16, 512])
Batch 5: Loss computed
Batch 5: Loss value 42.900482177734375
Batch 5: Backward pass done
Batch 5: Optimizer step done
Processing batch 6/52
Batch 6: Anchors shape torch.Size([16, 512])
Batc